In [1]:
import sqlite3 as sql
import pandas as pd

In [67]:
conn = sql.connect('pokedex.db')
c = conn.cursor()

def executeScriptsFromFile(filename):
    # Open and read the file as a single buffer
    fd = open(filename, 'r')
    sqlFile = fd.read()
    fd.close()

    # all SQL commands (split on ';')
    sqlCommands = sqlFile.split(';')

    # Execute every command from the input file
    for command in sqlCommands:
        c.execute(command)

In [68]:
executeScriptsFromFile('sql/criacao_tabelas.sql')

executeScriptsFromFile('sql/adc_pokemon.sql')
executeScriptsFromFile('sql/adc_tipos_pokemon.sql')
executeScriptsFromFile('sql/adc_ataques.sql')
executeScriptsFromFile('sql/adc_uso_ataque.sql')
executeScriptsFromFile('sql/adc_habilidades.sql')
executeScriptsFromFile('sql/adc_uso_habilidades.sql')
executeScriptsFromFile('sql/adc_uso_pokemon.sql')
executeScriptsFromFile('sql/add_evolucoes.sql')
executeScriptsFromFile('sql/adc_aliados.sql')

In [61]:
# Consulta 3.1 (Junção de 3 ou mais Relações)
# Lista de Habilidades por Pokemon, em ordem de uso (mais usadas às menos usadas)
# (dados de porcentagem de uso do nível OU de batalha da Smogon)
consulta3_1 = pd.read_sql_query('SELECT p.nome, h.nome, h.descricao'
                                + ' FROM (Habilidade AS h NATURAL JOIN UsoHabilidade AS uh)'
                                + ' JOIN Pokemon AS p ON p.id_pokemon = uh.id_pokemon'
                                + ' GROUP BY p.id_pokemon, h.id_habilidade'
                                + ' ORDER BY p.id_pokemon ASC, uh.porcentagem DESC', conn)
consulta3_1

,nome,nome,descricao
0,Venusaur,Thick Fat,The Pokémon is protected by a layer of thick f...
1,Venusaur,Chlorophyll,Boosts the Pokémon’s Speed stat in harsh sunli...
2,Venusaur,Overgrow,Powers up Grass-type moves when the Pokémon’s ...
3,Charizard,Drought,Turns the sunlight harsh when the Pokémon ente...
4,Charizard,Tough Claws,Powers up moves that make direct contact.
5,Charizard,Solar Power,"Boosts the Sp. Atk stat in harsh sunlight, but..."
6,Charizard,Blaze,Powers up Fire-type moves when the Pokémon’s H...
7,Charizard,Illuminate,Raises the likelihood of meeting wild Pokémon ...
8,Wartortle,Rain Dish,The Pokémon gradually regains HP in rain.
9,Wartortle,Torrent,Powers up Water-type moves when the Pokémon’s ...


In [62]:
# Consulta 3.2 (Junção de 3 ou mais Relações)
# Lista de Ataques por Pokemon, em ordem de uso (mais usadas às menos usadas)
# (dados de porcentagem de uso do nível OU de batalha da Smogon)
consulta3_2 = pd.read_sql_query('SELECT p.nome, a.nome, a.descricao'
                                + ' FROM (Ataque AS a NATURAL JOIN UsoAtaque AS ua)'
                                + ' JOIN Pokemon AS p ON p.id_pokemon = ua.id_pokemon'
                                + ' GROUP BY p.id_pokemon, a.id_ataque'
                                + ' ORDER BY p.id_pokemon ASC, ua.porcentagem DESC', conn)
consulta3_2

,nome,nome,descricao
0,Venusaur,Synthesis,Restores HP (varies by time).
1,Venusaur,Growth,Raises the SPCL. ATK rating.
2,Venusaur,Hidden-power,The power varies with the POKéMON.
3,Venusaur,Sludge-bomb,An attack that may poison the foe.
4,Venusaur,Giga-drain,Steals 1/2 of the damage inflicted.
5,Venusaur,Sleep-powder,May cause the foe to fall asleep.
6,Venusaur,Earthquake,Tough but useless vs. flying foes.
7,Venusaur,Leech-seed,Steals HP from the foe on every turn.
8,Venusaur,Solar-beam,1st turn: Prepare 2nd turn: Attack
9,Venusaur,Leaf-storm,A storm of sharp leaves is whipped up. The att...


In [78]:
# Consulta 3.3 (Junção de 3 ou mais Relações)
# Lista de Pokemons que, em um dado mês e ano, já foram mais utilizados em batalha
# do que suas evoluções, e qual evolução à que se refere
# (dados de porcentagem de uso do nível OU de batalha da Smogon)
consulta3_3 = pd.read_sql_query('SELECT DISTINCT p1.nome, p2.nome'
                                + ' FROM (((Pokemon NATURAL JOIN UsoPokemon) AS p1 JOIN Evolucao as ev'
                                + ' ON p1.id_pokemon = ev.id_de) JOIN (Pokemon NATURAL JOIN UsoPokemon) as p2'
                                + ' ON p2.id_pokemon = ev.id_para)'
                                + ' WHERE p1.ano = p2.ano AND p1.mes = p2.mes'
                                + ' AND p1.porcentagem > p2.porcentagem', conn)
consulta3_3

,nome,nome
0,Bulbasaur,Ivysaur
1,Charmander,Charmeleon
2,Squirtle,Wartortle
3,Caterpie,Metapod
4,Weedle,Kakuna
5,Weedle,Beedrill
6,Kakuna,Beedrill
7,Pidgey,Pidgeotto
8,Rattata,Raticate
9,Ekans,Arbok


In [69]:
# Consulta 4.1 (Funções de Agregação):
# Retorna o Pokemon mais utilizado em batalhas durante todo o tempo de 2014 a 2019
# (do nível OU de batalha da Smogon)
consulta4_1 = pd.read_sql_query('SELECT nome'
                         + ' FROM Pokemon NATURAL JOIN UsoPokemon'
                         + ' WHERE porcentagem = '
                         + '(SELECT MAX(porcentagem) FROM UsoPokemon)', conn)
consulta4_1

,nome
0,Landorus-incarnate


In [ ]:
# Consulta 4.1 (Funções de Agregação):
# Retorna uma lista com os anos e os tipos utilizados em batalhas,
# agrupados por ano (decrescente) por ordem de uso (do mais utilizado ao menos utilizado daquele ano)
# (do nível OU de batalha da Smogon)
consulta4_2 = pd.read_sql_query('SELECT ano, tipo'
                                + ' FROM UsoPokemon NATURAL JOIN Pokemon NATURAL JOIN TipoPokemon'
                                + ' GROUP BY ano, tipo'
                                + ' ORDER BY ano DESC, AVG(porcentagem) DESC', conn)
consulta4_2